In [41]:
import os
import re

import django
import fs.path
import numpy as np
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS
from more_itertools import sample

if not os.getcwd().endswith('multidex'):
    os.chdir('..')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import MSpec, ZSpec
from plotter.components import mspec_graph_line
from multidex_utils import model_metadata_df, djget
from plotter.graph import make_mspec_browse_image_components

In [3]:
mspec = MSpec.objects.all()[88]

In [13]:
%%time 
make_mspec_browse_image_components(
    mspec, "assets/browse/mcam/", "/static/"
)

CPU times: user 930 µs, sys: 3.45 ms, total: 4.38 ms
Wall time: 2.72 ms


Div([Img(id='spec-image-left', src='/static/sol0812_mcam03564_L126_ROIsOVERLAY_browse.jpg', style={'width': '50%', 'height': '50%'}), Img(id='spec-image-right', src='/static/sol0812_mcam03564_R126_ROIsOVERLAY_browse.jpg', style={'width': '50%', 'height': '50%'})])

In [25]:
%%time
mspec = djget(
    MSpec, 33, "id", "get"
)

CPU times: user 8.61 ms, sys: 484 µs, total: 9.09 ms
Wall time: 6.92 ms


In [24]:
%time MSpec.objects.all()[5]

CPU times: user 1.59 ms, sys: 2.92 ms, total: 4.51 ms
Wall time: 3.23 ms


<MSpec: sol1373_Uis_mcam06720>

In [ ]:
%%time 
mspec.filter_values(
    scale_to=('l1', 'r1'), 
    average_filters=True,
)
 

In [61]:
from plotter.components import GRAPH_DISPLAY_DEFAULTS, AXIS_DISPLAY_DEFAULTS
import plotly.graph_objects as go
from plotter.spectrum_ops import d2r

In [ ]:
scale_to = ('l1', 'r1')
average_filters = True
r_star = True
spectrum = mspec
%time spectrum_data = spectrum.filter_values(scale_to=scale_to, average_filters=average_filters)
x_axis = [filt_value["wave"] for filt_value in spectrum_data.values()]
y_axis = [filt_value["mean"] for filt_value in spectrum_data.values()]
y_error = [filt_value["err"] for filt_value in spectrum_data.values()]
# TODO: this definitely shouldn't be happening here
if r_star:
    if spectrum.incidence_angle:
        cos_theta_i = np.cos(d2r(spectrum.incidence_angle))
        y_axis = [mean / cos_theta_i for mean in y_axis]
        y_error = [err / cos_theta_i for err in y_error]
%time text = [filt + ", " + str(spectrum_data[filt]["wave"]) for filt in spectrum_data]
show_error=True

In [114]:
GRAPH_DISPLAY_DEFAULTS

{'margin': {'l': 10, 'r': 10, 't': 25, 'b': 0},
 'plot_bgcolor': 'rgba(0,0,50,0.05)',
 'paper_bgcolor': 'rgba(254,252,245,1)'}

In [131]:
%%time
fig = go.Figure(
    layout= {
        **GRAPH_DISPLAY_DEFAULTS,
        'xaxis': AXIS_DISPLAY_DEFAULTS | {"title_text": "wavelength"},
        'yaxis': AXIS_DISPLAY_DEFAULTS | {
            "title_text": "reflectance",
            "range": [0, min(y_axis) + max(y_axis)]
        },
    }
)
scatter = go.Scatter(
        x=x_axis,
        y=y_axis,
        mode="lines+markers",
        text=text,
        line={"color": spectrum.roi_hex_code()},
        error_y={"array": y_error, "visible": show_error},
    )
fig.add_trace(scatter)

CPU times: user 9.1 ms, sys: 0 ns, total: 9.1 ms
Wall time: 8.69 ms


In [111]:
fig = go.Figure(
    layout= {
        'xaxis': AXIS_DISPLAY_DEFAULTS | {"title_text": "wavelength"},
        'yaxis': AXIS_DISPLAY_DEFAULTS | {"title_text": "reflectance"}
    }
)

In [87]:
%time fig.add_trace(scatter)

CPU times: user 1.48 ms, sys: 0 ns, total: 1.48 ms
Wall time: 1.5 ms


In [93]:
fig.layout

Layout({
    'margin': {'b': 0, 'l': 10, 'r': 10, 't': 25},
    'paper_bgcolor': 'rgba(254,252,245,1)',
    'plot_bgcolor': 'rgba(0,0,50,0.05)',
    'template': '...',
    'xaxis': {'gridcolor': 'rgba(0,0,50,0.05)',
              'linecolor': 'rgba(0,0,25,0.18)',
              'linewidth': 2,
              'mirror': True,
              'showgrid': True,
              'showline': True,
              'spikecolor': 'rgba(0,0,25,0.18)',
              'tickcolor': 'rgba(36,28,1,1)',
              'tickfont': {'family': 'Fira Mono'},
              'title': {'font': {'family': 'Fira Mono'}, 'text': 'wavelength'},
              'zerolinecolor': 'rgba(0,0,25,0.18)'},
    'yaxis': {'gridcolor': 'rgba(0,0,50,0.05)',
              'linecolor': 'rgba(0,0,25,0.18)',
              'linewidth': 2,
              'mirror': True,
              'range': [0, 0.2790598924687113],
              'showgrid': True,
              'showline': True,
              'spikecolor': 'rgba(0,0,25,0.18)',
              't

In [90]:
# noinspection PyTypeChecker
%time fig.update_layout(GRAPH_DISPLAY_DEFAULTS)
%time fig.update_xaxes(AXIS_DISPLAY_DEFAULTS | {"title_text": "wavelength"})
%time fig.update_yaxes(AXIS_DISPLAY_DEFAULTS | {"title_text": "reflectance"})
%time fig.update_layout({"yaxis": {"range": [0, min(y_axis) + max(y_axis)]}})

CPU times: user 1.64 ms, sys: 281 µs, total: 1.92 ms
Wall time: 1.93 ms
CPU times: user 3.25 ms, sys: 0 ns, total: 3.25 ms
Wall time: 3.26 ms
CPU times: user 2.88 ms, sys: 0 ns, total: 2.88 ms
Wall time: 2.9 ms
CPU times: user 579 µs, sys: 0 ns, total: 579 µs
Wall time: 582 µs


In [ ]:
for spec in ZSpec.objects.all():
    spec.delete()

In [ ]:
input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/zcam")
output_fs = OSFS('.')
output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [ ]:
def reg_seq_id(seq_id_str):
    seq_id_str = seq_id_str.replace(" ", "")
    return 'zcam' + format(int(seq_id_str.split('zcam')[1]), "0>5d")

metaframe = pd.read_csv(input_fs.getsyspath('Metadata-marslab.csv'))
metaframe.columns = [column.lower() for column in metaframe.columns]
# add NaNs back in so we can programmatically delete them
metaframe.replace('-', np.nan, inplace=True)
# we're turning these to ints when we ingest them,
# but python doesn't like statements like int('3.0'),
# so turn to float as an intermediate step
for column in [
    'sol', 'site', 'drive', 'rover_elevation', 'target_elevation', 'tau',
    'focal_distance', 'incidence_angle', 'emission_angle',
    'phase_angle', 'l_s', 'site', 'drive', 'lat', 'lon', 'odometry',
]:
    metaframe[column] = metaframe[column].astype('float')
metaframe['ltst'] = metaframe['ltst'].astype('datetime64')
metaframe['seq_id'] = metaframe['seq_id'].map(reg_seq_id)
metaframe.loc[
    metaframe['sclk'].notna(), 'sclk'
] = metaframe.loc[metaframe['sclk'].notna(), 'sclk'].astype(float).astype(int)

In [ ]:
temp_images = [
    image for image in input_fs.listdir('temp_images')
]

In [ ]:
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('-marslab.csv') and file.startswith('Sol'))
]
for spec_file in spec_files:
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    observation = metaframe.loc[metaframe['seq_id'] == frame['seq_id'][0]]
    # hacky hacky -- but temporary
    if len(observation) > 1: 
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        observation = observation.loc[observation['name'].str.contains(str(pointing))]
    assert len(observation == 1)
    for _, row in frame.iterrows():
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in ZSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])
        # because we're manually associating
        row.drop(['sol', 'seq_id', 'instrument'], inplace=True)
        metadata = dict(row) | observation.iloc[0].dropna().to_dict() | {
            'filename': spec_file
        }
        spectrum = ZSpec(**metadata)
        spectrum.clean()
        spectrum.save()

In [ ]:
len(ZSpec.objects.all())

In [ ]:
output_image_dir

In [ ]:
z

In [ ]:
spec.seq_id

In [ ]:
os.listdir(output_image_dir)

In [ ]:
spec.filename

In [ ]:
import re

In [ ]:
for spec in ZSpec.objects.all():
    images = [
        image for image in os.listdir(output_image_dir + '/roi_browse')
        if spec.seq_id in image
    ]
    if 'pointing' in spec.filename.lower():
        pointing = re.search(r"(?<=Pointing)\d", spec_file).group()
        images = [image for image in images if 'pointing' + pointing in image]
    image_dict = {}
    for image in images:
        if 'rgb' in image:
            image_dict['rgb'] = image
        if 'enhanced' in image:
            image_dict['enhanced'] = image
    spec.images = str(image_dict)
    spec.clean()
    spec.save()

In [ ]:
from random import choice

In [ ]:
spec = choice(ZSpec.objects.all())

In [ ]:
from plotter.spectrum_ops import *

In [ ]:
unc = """L0R:3.3
L0G:3.3
L0B:3.7
L1:1.4
L2:1.1
L3:0.2
L4:1.8
L5:1.6
L6:0.4
R0R:3.7
R0G:4.1
R0B:4.6
R1:0.4
R2:0.3
R3:0.6
R4:0.5
R5:0.8
R6:0.4""".split("\n")

In [ ]:
unc

In [ ]:
{
    pair.split(":")[0]: float(pair.split(":")[1])
    for pair in unc
}

In [ ]:
filter_df = filter_df_from_queryset(
    ZSpec.objects.all(),
    average_filters=False,
    scale_to=None,
    r_star=True,
    )

In [ ]:
from marslab.compatibility import INSTRUMENT_UNCERTAINTIES
from itertools import chain, product

In [ ]:
spec_model = ZSpec
instrument = spec_model.instrument
unc = INSTRUMENT_UNCERTAINTIES[instrument]
function = ratio
# filters = ['L5', 'R3', 'L4']
filters = ['L5', 'R3']

In [ ]:
def compute_minmax_spec_error(filter_df, spec_model, spec_op, *filters):
    # cartesian product of these sets gives all possible sign combos for error high, error low,
    # i.e., crude bounds for the hull of the range of possible measurements
    unc = INSTRUMENT_UNCERTAINTIES[spec_model.instrument]
    corners = product(*[[1, -1] for filt in filters])
    bounds_df_list = []
    # apply these signs to uncertainty values, getting a list of dataframes
    # giving values of all measurements in set at the upper / lower bound combinations
    # for uncertainties associated with each relevant filter.
    for corner in corners:
        corner_series_list = []
        for filt_ix, sign in enumerate(corner):
            filt = filters[filt_ix]
            corner_series_list.append(
                filter_df[filt]  + filter_df[filt] * corner[filt_ix] * unc[filt] / 100
            )
        corner_df = pd.concat(corner_series_list, axis=1)
        # record the value of the spectrum op for each of these bounding dataframes.
        bounds_df_list.append(function(corner_df, spec_model, *filters)[0])   
    # compute the nominal value and compare it to values at these bounds
    possible_values = pd.concat(bounds_df_list, axis=1)
    nominal_value = function(filter_df, spec_model, *filters)[0]
    offsets = possible_values.sub(nominal_value, axis=0) 
    # then min / max of each of these gives us an error estimate for each spectrum
    return offsets.min(axis=1), offsets.max(axis=1)

In [ ]:
# cartesian product of these sets gives all possible sign combos for error high, error low,
# i.e., crude bounds for the hull of the range of possible measurements
corners = product(*[[1, -1] for filt in filters])
bounds_df_list = []
# apply these signs to uncertainty values, getting a list of dataframes
# giving values of all measurements in set at the upper / lower bound combinations
# for uncertainties associated with each relevant filter.
for corner in corners:
    corner_series_list = []
    for filt_ix, sign in enumerate(corner):
        filt = filters[filt_ix]
        corner_series_list.append(
            filter_df[filt]  + filter_df[filt] * corner[filt_ix] * unc[filt] / 100
        )
    corner_df = pd.concat(corner_series_list, axis=1)
    # record the value of the spectrum op for each of these bounding dataframes.
    bounds_df_list.append(function(corner_df, spec_model, *filters)[0])    

In [ ]:
# compute the nominal value and compare it to values at these bounds
possible_values = pd.concat(bounds_df_list, axis=1)
nominal_value = function(filter_df, spec_model, *filters)[0]
offsets = possible_values.sub(nominal_value, axis=0) 
# then min / max of each of these gives us an error estimate for each spectrum
error_low = offsets.min(axis=1)
error_high = offsets.max(axis=1)

In [ ]:
nominal_value = function(filter_df, spec_model, *filters)[0]

In [ ]:
possible_values

In [ ]:
nominal_value

In [ ]:
nominal_value = function(filter_df, spec_model, *filters)[0]
bound = function(bounds_df_list[0], spec_model, *filters)[0] 
bound - nominal_value

In [ ]:
# note that band_depth_min is a special case

nominal_value = band_avg(filter_df, ZSpec, 'L1', 'R1')[0]
band_avg(bounds_df_list[0], ZSpec, 'L1', 'R1')[0] - nominal_value

In [ ]:
corner_df - nominal_value

In [ ]:
list(product(filter_df_low.columns, filter_df_high))

In [ ]:
filter_df_low